In [12]:
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    fbeta_score,
)

df = pd.read_csv("../stats/combined.csv")
X = df.drop(["chr", "start", "end", "cnv_type", "BAM_CREF_SKIP", "BAM_CSOFT_CLIP", "BAM_CHARD_CLIP", "BAM_CPAD", "BAM_CEQUAL", "BAM_CDIFF", "BAM_CBACK"], axis=1)
lbl_e = LabelEncoder()
y = lbl_e.fit_transform(df["cnv_type"])

# Podział na zbiór treningowy i testowy
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
#Define the objective function to optimize
results = []
def objective(trial):
    # Define the hyperparameters to search over
    model_type = trial.suggest_categorical('model_type', ['RandomForest', 'LightGBM', 'XGBoost'])
    n_estimators = trial.suggest_int('n_estimators', 10, 100, step=10)
    max_depth = trial.suggest_int('max_depth', 10, 100, step=10)
    class_weight = trial.suggest_categorical('class_weight', [None, 'balanced', {0: 1, 1: 3, 2: 3}])
    log_transform = trial.suggest_categorical('log_transform', [True, False])
    standard_scaler = trial.suggest_categorical('standard_scaler', [True, False])
    undersampling = trial.suggest_categorical('undersampling', [True, False])

    # Preprocess the data based on hyperparameters
    if undersampling:
        # Undersampling klas mniejszościowych (przykład)
        count = Counter(y_train)
        classes_resampling = {2: int(count[2] * 0.3)}
        under = RandomUnderSampler(
            sampling_strategy=classes_resampling, random_state=42
        )
        X_res, y_res = under.fit_resample(X_train, y_train)
    else:
        X_res, y_res = X_train, y_train

    if standard_scaler:
        scaler = StandardScaler()
        X_res = scaler.fit_transform(X_res)
        x_test_res = scaler.transform(x_test)
    
    if log_transform:
        X_res = np.log1p(X_res)
        x_test_res = np.log1p(x_test)


    if model_type == 'RandomForest':
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, class_weight=class_weight, random_state=42, n_jobs=-1)
    elif model_type == 'LightGBM':
        model = LGBMClassifier(n_estimators=n_estimators, max_depth=max_depth, class_weight=class_weight, random_state=42, n_jobs=-1)
    elif model_type == 'XGBoost':
        model = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, scale_pos_weight=class_weight, random_state=42, n_jobs=-1)


    # Trenowanie modelu
    model.fit(X_res, y_res)

    # Przewidywanie na zbiorze testowym
    if standard_scaler or log_transform:
        y_pred = model.predict(x_test_res)
    else:
        y_pred = model.predict(x_test)
    fbeta = fbeta_score(y_test, y_pred, beta = 3, average='macro')


    results.append({
        'model': model_type,
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'class_weight': class_weight,
        'log_transform': log_transform,
        'standard_scaler': standard_scaler,
        'undersampling': undersampling,
        'fbeta': fbeta,
        'classification_report': classification_report(y_test, y_pred, zero_division=True),
        'confusion_matrix': confusion_matrix(y_test, y_pred)
    })

    return fbeta

# Load your data and preprocess it
# ...

# Split the data into training and testing sets
# ...

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Get the best hyperparameters
best_params = study.best_params
best_accuracy = study.best_value

print("Najlepsze hiperparametry:")
print(best_params)
print("Najlepsza dokładność:", best_accuracy)


[I 2023-10-10 01:12:55,673] A new study created in memory with name: no-name-4ffb36d8-dd42-4026-b923-26b4ad45a132
/Users/blewandowski/CNV-detection-with-ML/cnv-detection/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 3, 2: 3} which is of type dict.
  warnings.warn(message)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032256 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1768
[LightGBM] [Info] Number of data points in the train set: 4004094, number of used features: 9
[LightGBM] [Info] Start training from score -2.282466
[LightGBM] [Info] Start training from score -1.173089
[LightGBM] [Info] Start training from score -0.530080
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


[I 2023-10-10 01:13:09,469] Trial 0 finished with value: 0.22361611448493712 and parameters: {'model_type': 'LightGBM', 'n_estimators': 60, 'max_depth': 30, 'class_weight': {0: 1, 1: 3, 2: 3}, 'log_transform': True, 'standard_scaler': True, 'undersampling': True}. Best is trial 0 with value: 0.22361611448493712.
/Users/blewandowski/CNV-detection-with-ML/cnv-detection/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 3, 2: 3} which is of type dict.
  warnings.warn(message)
/var/folders/g6/1vkdlq6j11bg56107lyg615w0000gp/T/ipykernel_42018/1018974133.py:31: RuntimeWarning: invalid value encountered in log1p
  X_res = np.log1p(X_res)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057211 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1758
[LightGBM] [Info] Number of data points in the train set: 8570985, number of used features: 9
[LightGBM] [Info] Start training from score -3.146749
[LightGBM] [Info] Start training from score -2.037372
[LightGBM] [Info] Start training from score -0.190389
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


[I 2023-10-10 01:13:27,388] Trial 1 finished with value: 0.24116456895847774 and parameters: {'model_type': 'LightGBM', 'n_estimators': 50, 'max_depth': 20, 'class_weight': {0: 1, 1: 3, 2: 3}, 'log_transform': True, 'standard_scaler': True, 'undersampling': False}. Best is trial 1 with value: 0.24116456895847774.
/Users/blewandowski/CNV-detection-with-ML/cnv-detection/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 3, 2: 3} which is of type dict.
  warnings.warn(message)


...

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031785 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1765
[LightGBM] [Info] Number of data points in the train set: 4004094, number of used features: 9
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


[I 2023-10-10 01:45:34,762] Trial 47 finished with value: 0.8813324097750904 and parameters: {'model_type': 'LightGBM', 'n_estimators': 40, 'max_depth': 70, 'class_weight': 'balanced', 'log_transform': True, 'standard_scaler': False, 'undersampling': True}. Best is trial 30 with value: 0.8838367214151394.
/Users/blewandowski/CNV-detection-with-ML/cnv-detection/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 1, 1: 3, 2: 3} which is of type dict.
  warnings.warn(message)
[I 2023-10-10 01:46:21,897] Trial 48 finished with value: 0.880851006346771 and parameters: {'model_type': 'RandomForest', 'n_estimators': 60, 'max_depth': 30, 'class_weight': None, 'log_transform': True, 'standard_scaler': False, 'undersampling': True}. Best is trial 30 with value: 0.8838367214151394.
/Users/blewandowski/CNV-detection-with-ML/cnv-detection/lib/python3.

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.058177 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1755
[LightGBM] [Info] Number of data points in the train set: 8570985, number of used features: 9
[LightGBM] [Info] Start training from score -2.130622
[LightGBM] [Info] Start training from score -2.119857
[LightGBM] [Info] Start training from score -0.272875
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


[I 2023-10-10 01:46:46,237] Trial 49 finished with value: 0.8625457333638478 and parameters: {'model_type': 'LightGBM', 'n_estimators': 70, 'max_depth': 50, 'class_weight': None, 'log_transform': True, 'standard_scaler': False, 'undersampling': False}. Best is trial 30 with value: 0.8838367214151394.


Najlepsze hiperparametry:
{'model_type': 'LightGBM', 'n_estimators': 60, 'max_depth': 50, 'class_weight': None, 'log_transform': True, 'standard_scaler': False, 'undersampling': True}
Najlepsza dokładność: 0.8838367214151394


In [15]:
pd.DataFrame(results).to_csv("results.csv", index=False)